# Parsing training set

https://github.com/conor-horgan/DeepeR has a link to a dataset of spectra. 

The database is composed of paired examples (noisy and high SNR versions of the same spectra)

Here, we parse this dataset to train our unsupervised spectral denoising framework.

In [1]:
# .mat files are nto in hdf5 format for some reason, so we use scipy.io to read them...
import numpy as np
import scipy.io
import random

#set how much of the training data to split for train and valid
train_split = 0.8
validation_split = 0.2
assert((train_split + validation_split) < 1)
#use this if we'd like plit across the combined set formed by their train/test files
#test_split = 1 - (train_split + validation_split)

hn_train = scipy.io.loadmat('Train_Inputs.mat')
hn_train = hn_train['Train_Inputs']
ln_train = scipy.io.loadmat('Train_Outputs.mat')
ln_train = ln_train['Train_Outputs']

hn_test = scipy.io.loadmat('Test_Inputs.mat')
hn_test = hn_test['Test_Inputs']
ln_test = scipy.io.loadmat('Test_Outputs.mat')
ln_test = ln_test['Test_Outputs']

In [2]:
#use this if we'd like plit across the combined set formed by their train/test files
#hn_all = np.concatenate((hn_train,hn_test), axis=0)
#ln_all = np.concatenate((ln_train,ln_test), axis=0)

In [3]:
hn_all = hn_train
ln_all = ln_train

(12694, 500)
(159618, 500)


In [5]:
randomised_indices = random.sample(range(0, np.round(np.shape(hn_all)[0])), int(np.floor(np.shape(hn_all)[0])))
train_indices = randomised_indices[:int(np.round(train_split*np.shape(hn_all)[0]))]
valid_indices = randomised_indices[int(np.round(train_split*np.shape(hn_all)[0])):(int(np.round(train_split*np.shape(hn_all)[0]))+int(np.round(validation_split*np.shape(hn_all)[0])))]
#use this if we'd like plit across the combined set formed by their train/test files
#test_indices = randomised_indices[(int(np.round(train_split*np.shape(hn_all)[0]))+int(np.round(validation_split*np.shape(hn_all)[0]))):]


In [6]:
hn_train_set = hn_all[train_indices[:int(round(len(train_indices)/2))]]
ln_train_set = ln_all[train_indices[int(round(len(train_indices)/2)):]]

#validation set and test set is paired data
hn_valid_set = hn_all[valid_indices]
ln_valid_set = ln_all[valid_indices]
#We just use theor original test set as our test set
hn_test_set = hn_all[test_indices]
ln_test_set = ln_all[test_indices]

In [7]:
#for now, we just use the etst set as validation... but this will change in the future. 
hn_valid_set = hn_test
ln_valid_set = ln_test

In [8]:
np.save('hn_train_set.npy', hn_train_set)
np.save('ln_train_set.npy', ln_train_set)

np.save('hn_valid_set.npy', hn_valid_set)
np.save('ln_valid_set.npy', ln_valid_set)

np.save('hn_test_set.npy', hn_test_set)
np.save('ln_test_set.npy', ln_test_set)

In [9]:
#test whether any indices are shared between datasets

print(np.max(np.in1d(train_indices,valid_indices)))
print(np.max(np.in1d(train_indices,test_indices)))
print(np.max(np.in1d(valid_indices,test_indices)))

False
False
False


In [10]:
hn = train_indices[:int(round(len(train_indices)/2))]

In [11]:
ln = train_indices[int(round(len(train_indices)/2)):]

In [12]:
print(np.max(np.in1d(hn,ln)))

False


True
